In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

import torch 
torch.cuda.current_device()

Sat Apr 27 07:57:37 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:01:00.0 Off |                  Off |
| 57%   79C    P2            240W /  300W |   48668MiB /  49140MiB |     74%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

0

In [5]:
print(torch.__version__)

2.0.1+cu117


In [6]:
from datasets import load_dataset, load_metric
from datasets import ClassLabel

In [8]:
import datasets
print(datasets.__version__)

2.16.1


In [9]:
import transformers
print(transformers.__version__)

4.36.2


In [7]:
from datasets import load_dataset, load_metric
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

timit = load_dataset("timit_asr", data_dir='./timit/data/lisa/data/timit/raw/TIMIT')
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

## text normalization    
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

timit = timit.map(remove_special_characters)
show_random_elements(timit["train"].remove_columns(["audio", "file"]))

/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/datasets/load.py:922: FutureWarning: The repository for timit_asr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at timit_asr/timit_asr.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4620 [00:00<?, ? examples/s]

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

,text
0,don't ask me to carry an oily rag like that
1,don't ask me to carry an oily rag like that
2,behind him his mates stirred without waking
3,they should live in modest circumstances avoiding all conspicuous consumption
4,too much curiosity can get you into trouble
5,they all agree that the essay is barely intelligible
6,they often go out in the evening
7,the proof that you are seeking is not available in books
8,don't ask me to carry an oily rag like that
9,dolphins are intelligent marine mammals


In [8]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

## instantiate an object of the Wav2Vec2CTCTokenizer class.
import json
from transformers import Wav2Vec2CTCTokenizer

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Map:   0%|          | 0/4620 [00:00<?, ? examples/s]

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [9]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [10]:
import IPython.display as ipd
import numpy as np
import random

def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)

# trim out audio sequences that are longer than 4sec. 
max_input_length_in_sec = 4.0
timit["train"] = timit["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Map (num_proc=4):   0%|          | 0/4620 [00:00<?, ? examples/s]

/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated

Map (num_proc=4):   0%|          | 0/1680 [00:00<?, ? examples/s]

/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated

Filter:   0%|          | 0/4620 [00:00<?, ? examples/s]

In [11]:
from transformers import Wav2Vec2ForCTC

pred_processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo")
finetuned_model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo").to("cuda")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/transformers/configuration_utils.py:387: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

In [12]:
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

/tmp/ipykernel_3339598/3447755518.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/home/ganesh/translation_improvement_pipeline/a_venv/lib/python3.8/site-packages/datasets/load.py:752: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [13]:
import torch 

def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = finetuned_model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = pred_processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)

    return batch

# decode Timit test set 
results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

In [14]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

show_random_elements(results)

Test WER: 0.186


,pred_str,text
0,they clothed sometime after eight,they close sometime after eight
1,swing your arm as high as you can,swing your arm as high as you can
2,the clumsy coustumer spilled some expensive perfume,the clumsy customer spilled some expensive perfume
3,each untimely income loss coincided with the break down of a heating system part,each untimely income loss coincided with the breakdown of a heating system part
4,a chosen few will become generals,a chosen few will become generals
5,right now may not be the best time for busines murgers,right now may not be the best time for business mergers
6,students choices of ideal educational gols are not arbetrary or wheymsical,students' choices of ideal educational goals are not arbitrary or whimsical
7,she had your dark suit in greasy wash water all year,she had your dark suit in greasy wash water all year
8,out would be a collossealashame to throuw away a story like this,it would be a colossal shame to throw away a story like this
9,the cranberry bog gets very perty in autum,the cranberry bog gets very pretty in autumn


In [15]:
finetuned_model.wav2vec2

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

In [16]:
import torch.nn.utils.prune as prune

def pruning_bert(model, px, model_type='wav2vec_small'):
    """
    prune out wav2vec 2.0 BERT: 12 transformer layers for BASE, and 24 
                                transformer layers for LARGE

    note: position encoding, projection heads, layernorm statistics are not pruned. 
    """
    if model_type == 'wav2vec_small':
        num_transformer_blocks = 12
    elif model_type == 'libri960_big' or model_type == 'xlsr_53_56k':
        num_transformer_blocks = 24
    else:
        print('model type {} not supported'.format(model_type))        
    print('num_transformer_blocks is', num_transformer_blocks)

    parameters_to_prune =[]
    for ii in range(num_transformer_blocks):
        parameters_to_prune.append((model.encoder.layers[ii].attention.k_proj, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.k_proj, 'bias'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.v_proj, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.v_proj, 'bias'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.q_proj, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.q_proj, 'bias'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.out_proj, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].attention.out_proj, 'bias'))
        parameters_to_prune.append((model.encoder.layers[ii].feed_forward.intermediate_dense, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].feed_forward.intermediate_dense, 'bias'))
        parameters_to_prune.append((model.encoder.layers[ii].feed_forward.output_dense, 'weight'))
        parameters_to_prune.append((model.encoder.layers[ii].feed_forward.output_dense, 'bias'))

    parameters_to_prune = tuple(parameters_to_prune)

    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=px,
    )
        
def unprune_bert(model, model_type='wav2vec_small'):
    """
    remove pruning forward pre-hook. This is useful when we want to tweek the learned pruned mask, which is used in PARP.
    """
    if model_type == 'wav2vec_small':
        num_transformer_blocks = 12
    elif model_type == 'libri960_big' or model_type == 'xlsr_53_56k':
        num_transformer_blocks = 24
    else:
        print('model type {} not supported'.format(model_type))
    print('num_transformer_blocks is', num_transformer_blocks)

    parameters_to_prune =[]
    for ii in range(num_transformer_blocks):
        parameters_to_prune.append(model.encoder.layers[ii].attention.k_proj)
        parameters_to_prune.append(model.encoder.layers[ii].attention.v_proj)
        parameters_to_prune.append(model.encoder.layers[ii].attention.q_proj)
        parameters_to_prune.append(model.encoder.layers[ii].attention.out_proj)
        parameters_to_prune.append(model.encoder.layers[ii].feed_forward.intermediate_dense)
        parameters_to_prune.append(model.encoder.layers[ii].feed_forward.output_dense)

    for ii in range(0, len(parameters_to_prune)): # applying both weight+bias masks
        prune.remove(parameters_to_prune[ii], 'weight')
        prune.remove(parameters_to_prune[ii], 'bias')

def see_weight_rate(model, model_type='wav2vec_small'):
    """ check a model's zero rate 
    """
    if model_type == 'wav2vec_small':
        num_transformer_blocks = 12
    elif model_type == 'libri960_big' or model_type == 'xlsr_53_56k':
        num_transformer_blocks = 24
    else:
        print('model type {} not supported'.format(model_type))        
    print('num_transformer_blocks is', num_transformer_blocks)

    sum_list_2, zero_sum_2 = 0, 0
    for ii in range(num_transformer_blocks):
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.k_proj.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.k_proj.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.k_proj.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.k_proj.bias == 0))

        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.v_proj.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.v_proj.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.v_proj.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.v_proj.bias == 0))

        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.q_proj.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.q_proj.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.q_proj.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.q_proj.bias == 0))

        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.out_proj.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.out_proj.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].attention.out_proj.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].attention.out_proj.bias == 0))

        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].feed_forward.intermediate_dense.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].feed_forward.intermediate_dense.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].feed_forward.intermediate_dense.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].feed_forward.intermediate_dense.bias == 0))

        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].feed_forward.output_dense.weight.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].feed_forward.output_dense.weight == 0))
        sum_list_2 = sum_list_2 + float(model.encoder.layers[ii].feed_forward.output_dense.bias.nelement())
        zero_sum_2 = zero_sum_2 + float(torch.sum(model.encoder.layers[ii].feed_forward.output_dense.bias == 0))

    bert_zero_rate = 100 * zero_sum_2 / sum_list_2
    print('BERT zero rate is {0:.2f}'.format(bert_zero_rate))
    return bert_zero_rate

In [22]:
pruning_rate = 0.5
pruning_bert(finetuned_model.wav2vec2, pruning_rate, model_type='wav2vec_small')
see_weight_rate(finetuned_model.wav2vec2)

num_transformer_blocks is 12


OutOfMemoryError: CUDA out of memory. Tried to allocate 326.00 MiB (GPU 0; 47.53 GiB total capacity; 3.47 GiB already allocated; 8.06 MiB free; 3.86 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF